In [1]:
!pip install unidecode

In [2]:
# Library to suppress warnings or deprecation notes 
import warnings
warnings.filterwarnings('ignore')

import os
import math

# Libraries to help with reading and manipulating data
import numpy as np
import pandas as pd
import glob
from tqdm import tqdm

# Import the required library
from geopy.geocoders import Nominatim 
from unidecode import unidecode
import difflib 
import numpy as np

In [46]:
def match_one_powiat_in_pl(pl, row, clean_pl_col, clean_city_col, pl_col):
    result = None
    fallback_dict = {'FRANKFURT/ODER': 'SLUBICE', 'FRANKFURT/ODER': 'SLUBICE', 'BABICE': 'WARSZAWA',\
                    'KUZNIA NIEBOROWICKA': 'GLIWICE', 'GOZOW': 'GORZOW', 'LUBLINEK': 'LODZ',\
                    'CZOSTOCHOWA': 'CZESTOCHOWA'}
    if (row['county'] != row['county']):
        station = row['STANAME'].strip()
        city_village = station
        if ('-' in station):
            city_village = station.split('-')[0]
        elif (' ' in station):
            city_village = station.split(' ')[0]

        city_village = city_village.upper()

        # See if this is in fallback list of cities
        if (city_village in fallback_dict):
            city_village = fallback_dict[city_village]

        # Check in both clean_pl_col and clean_city_col
        powiat_mapping = pl[(pl[clean_city_col].str.startswith(city_village)) | \
                              (pl[clean_city_col].str.contains(city_village)) | \
                              (pl[clean_pl_col].str.startswith(city_village)) | \
                             (pl[clean_pl_col].str.startswith(city_village))][pl_col]

        if (powiat_mapping.size > 0):
            powiat_mapping = powiat_mapping.iloc[0] 
        else:
            powiat_mapping = np.NaN

        result = powiat_mapping 

    else:
        result = row['county']
    
    return result

def process_decode(x, key):
    return 'UNKNOWN' if x[key] != x[key] else unidecode(x[key]).upper()

In [47]:
air_filename = '../../../src/data/data_processed/merged_air_quality_data_CSV/air_quality_stations_with_powiat.csv'
air_df = pd.read_csv(air_filename)

In [5]:
air_df.head()

,No,Station code,International code,Station name,Old station code,Launching date,Closing date,Station type,Region type,Station model,Voivodeship,City,Address,WGS84 φ N,WGS84 λ E,county
0,1,DsBialka,NaN,Białka,DsBialka,1990-01-03,2005-12-31,industrial,suburban,container,DOLNOŚLĄSKIE,Białka,NaN,51.197783,16.117390,powiat legnicki
1,2,DsBielGrot,NaN,Bielawa - ul. Grota Roweckiego,DsBielGrot,1994-01-02,2003-12-31,background,urban,in-house,DOLNOŚLĄSKIE,Bielawa,ul. Grota Roweckiego 6,50.682510,16.617348,powiat dzierżoniowski
2,3,DsBogatFrancMOB,PL0602A,Bogatynia Mobil,DsBogatMob,2015-01-01,2015-12-31,background,urban,mobile,DOLNOŚLĄSKIE,Bogatynia,ul. Francuska/Kręta,50.940998,14.916790,powiat zgorzelecki
3,4,DsBogChop,PL0315A,Bogatynia - Chopina,DsBogChop,1996-01-01,2013-12-31,industrial,urban,container,DOLNOŚLĄSKIE,Bogatynia,ul. Chopina 35,50.905856,14.967175,powiat zgorzelecki
4,5,DsBogZatonieMob,PL0576A,Bogatynia - Mobil,DsBogZatonieMob,2012-01-01,2012-12-31,industrial,urban,mobile,DOLNOŚLĄSKIE,Bogatynia,"ul. Konrada, Zatonie",50.943245,14.913327,powiat zgorzelecki


In [6]:
air_df[air_df['county'].isnull()].shape[0]

450

In [7]:
air_df['City'] = np.where(air_df['City'].isnull(), air_df['Station name'], air_df['City'])

# Creation a decoded station name column for lookups
air_df['STANAME'] = air_df.apply(lambda row: unidecode(row.City).upper(), axis = 1)

In [8]:
pl_filename = '../../../src/data/data_auxiliary/PL.txt'
pl_df = pd.read_csv('c:/temp/PL.txt', sep="\t")

In [9]:
pl_df.head()

,country code,postal code,place name1,admin name1,admin code1,admin name2,admin code2,admin name3,admin code3,latitude,longitude,accuracy
0,PL,59-700,Rakowice,Lower Silesia,72,Powiat bolesławiecki,201.0,Bolesławiec,20102.0,51.2447,15.5347,6
1,PL,59-700,Chościszowice,Lower Silesia,72,Powiat bolesławiecki,201.0,Bolesławiec,20102.0,51.2868,15.5891,6
2,PL,59-700,Golnice,Lower Silesia,72,Powiat bolesławiecki,201.0,Bolesławiec,20102.0,51.3504,15.5852,6
3,PL,59-700,Dąbrowa Bolesławiecka,Lower Silesia,72,Powiat bolesławiecki,201.0,Bolesławiec,20102.0,51.3264,15.5772,6
4,PL,59-700,Parkoszów,Lower Silesia,72,Powiat bolesławiecki,201.0,Bolesławiec,20102.0,51.3753,15.5811,6


In [10]:
pl_df['admin_name2_clean'] = pl_df.apply(lambda row: process_decode(row, 'admin name2'), axis = 1)
pl_df['admin_name3_clean'] = pl_df.apply(lambda row: process_decode(row, 'admin name3'), axis = 1)

In [11]:
air_df['county'] = air_df.apply(lambda row: match_one_powiat_in_pl(pl_df, row, \
                                        'admin_name2_clean', 'admin_name3_clean', 'admin name2'), axis = 1)

In [12]:
air_df[air_df['county'].isnull()].shape[0]

3

In [13]:
# Write out the air pollution file with the mapped powiats
#air_output_filename = '../../../src/data/data_processed/merged_air_quality_data_CSV/thayes_air_quality_stations_with_powiat_missing3.csv'
#air_df.to_csv(air_output_filename ,index=False)

In [48]:
station_powait_filename = '../../../src/data/data_processed/processed_weather_data_in_CSV_files/stations_with_powiat_voivod.csv'
station_powiat_df = pd.read_csv(station_powait_filename)

In [49]:
station_powiat_df.head()

,Unnamed: 0,STAID,STANAME,CN,LAT,LON,HGHT,county,state
0,0,24860,PSZCZYNA,PL,50.000000,18.916667,261,powiat pszczyński,województwo śląskie
1,1,24876,JABLONKA,PL,49.466667,19.700000,671,powiat nowotarski,województwo małopolskie
2,2,24880,GUBALOWKA,PL,49.300000,19.933333,856,powiat tatrzański,województwo małopolskie
3,3,24881,WITOW,PL,49.333333,19.833333,844,powiat tatrzański,województwo małopolskie
4,4,24885,LAZY,PL,49.966667,20.500000,251,powiat bocheński,województwo małopolskie


In [58]:
station_powiat_df[station_powiat_df['county'].isnull()].shape[0]

0

In [51]:
station_powiat_df['county'] = station_powiat_df.apply(lambda row: match_one_powiat_in_pl(pl_df, row, \
                                        'admin_name2_clean', 'admin_name3_clean', 'admin name2'), axis = 1)

In [52]:
station_powiat_df[station_powiat_df['county'].isnull()].shape[0]

0

In [53]:
station_powiat_df[station_powiat_df['county'].isnull()]

,Unnamed: 0,STAID,STANAME,CN,LAT,LON,HGHT,county,state


In [57]:
station_powiat_df.head()

,STAID,STANAME,CN,LAT,LON,HGHT,county,state
0,24860,PSZCZYNA,PL,50.000000,18.916667,261,powiat pszczyński,województwo śląskie
1,24876,JABLONKA,PL,49.466667,19.700000,671,powiat nowotarski,województwo małopolskie
2,24880,GUBALOWKA,PL,49.300000,19.933333,856,powiat tatrzański,województwo małopolskie
3,24881,WITOW,PL,49.333333,19.833333,844,powiat tatrzański,województwo małopolskie
4,24885,LAZY,PL,49.966667,20.500000,251,powiat bocheński,województwo małopolskie


In [59]:
# Write out the station powiat file with the mapped powiats
#station_powiat_df.to_csv(station_powait_filename, index=False)